# CNN example with Keras

In this example, we classify hand-written digits of the MNIST dataset with LeNet5. Please read CNN example with Chainer for the detail of the procedure, the dataset, and the architecture. This example is based on the official example in Keras.

## Basics of Keras

### Model

**keras.engine.Layer**

* Building block of models.
* Parameterized functions (e.g. ``Dense``, ``Convolution2D``), activation functions (instances of ``Activation`` class) or shape transformations (e.g. ``Flatten``) are the examples of ``Layer`` class.

**keras.model.Sequential**

* Container of layers.
* Users sequentially add layers with ``Sequential.add()`` method.


### Backend

Keras relies on third-party libraries for low-level operations on tensors. Currently, Theano and TensorFlow are available as backends. Users can switch the backend by several ways (see [the official instruction](http://keras.io/backend/) for details.)

## Codes

### 1. Import packages

In [1]:
from __future__ import print_function

import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils

Using Theano backend.


"dim_ordering" determine how an image is represented as an 4-dimensional array.

* dim_ordering = 'th' (THeano)    : (batchsize, channels, rows, columns)
* dim_ordering = 'tf' (TensorFlow): (batchsize, rows, columns, channels)

In [2]:
from keras import backend as K
print(K._BACKEND)
if K.image_dim_ordering() == 'tf':
    K.set_image_dim_ordering('th')

theano


### 2. Prepare dataset 

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [4]:
# Set GPU mode
gpu = 0
if gpu >= 0:
    from theano.sandbox import cuda
    cuda.use("gpu{}".format(gpu))

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
/home/delta/.pyenv/versions/anaconda3-2.5.0/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


### 3. Prepare model and 4. Setup optimizer

In [6]:
model = Sequential()

# The size of each image of MNIST is 28 x 28,
# but LeNet5 is designed so that its input is 32 x 32.
# So we add the padding of size (32 - 28) / 2 = 2.
model.add(ZeroPadding2D(padding=(2, 2), input_shape=(1, 28, 28)))
model.add(Convolution2D(6, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(16, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120))
model.add(Activation('relu'))
model.add(Dense(84))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

### 5. Training

In [7]:
model.fit(X_train, Y_train, batch_size=128, nb_epoch=12,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 2s - loss: 0.3651 - acc: 0.8881 - val_loss: 0.0858 - val_acc: 0.9731
Epoch 2/12
60000/60000 [==============================] - 2s - loss: 0.0839 - acc: 0.9744 - val_loss: 0.0608 - val_acc: 0.9809
Epoch 3/12
60000/60000 [==============================] - 2s - loss: 0.0623 - acc: 0.9814 - val_loss: 0.0472 - val_acc: 0.9846
Epoch 4/12
60000/60000 [==============================] - 2s - loss: 0.0513 - acc: 0.9843 - val_loss: 0.0421 - val_acc: 0.9859
Epoch 5/12
60000/60000 [==============================] - 2s - loss: 0.0442 - acc: 0.9867 - val_loss: 0.0412 - val_acc: 0.9861
Epoch 6/12
60000/60000 [==============================] - 2s - loss: 0.0390 - acc: 0.9881 - val_loss: 0.0338 - val_acc: 0.9892
Epoch 7/12
60000/60000 [==============================] - 2s - loss: 0.0348 - acc: 0.9893 - val_loss: 0.0350 - val_acc: 0.9878
Epoch 8/12
60000/60000 [==============================] - 2s 

In [8]:
# Evaluation (optional)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0291142435878
Test accuracy: 0.9904


### 6. Save models 

In [9]:
with open('lenet5.json', 'w') as o:
    o.write(model.to_json())
model.save_weights('lenet5.h5')